In [611]:
import json
from decimal import Decimal, getcontext
from datetime import datetime
from web3 import Web3

getcontext().prec = 18

In [612]:
CATS_CONTRACT = "0xF084962cdC640ED5c7d4e35E52929dAC06B60F7C"
CATS_RESERVE = "0x4eA682B94B7e13894C3d0b9afEbFbDd38CdACc3C"

w3 = Web3(Web3.HTTPProvider("https://rpc.blast.io"))
assert w3.is_connected(), "Failed to connect to Blast"

In [613]:
with open("./PlutocatsToken.json", "r") as f:
    PLUTOCATS_ABI = json.loads(f.read())
    
contract = w3.eth.contract(address=CATS_CONTRACT, abi=PLUTOCATS_ABI)

In [614]:
def to_eth(wei):
    return round(float(Web3.from_wei(wei, 'ether')),8)

def date_to_unix(date_str):
    date_obj = datetime.strptime(date_str, '%Y-%m-%d')
    unix_time = int(date_obj.timestamp())
    return unix_time

def format_time_units(time_in_hours):
    days = time_in_hours // 24
    hours = time_in_hours % 24
    result = ""
    if days > 0:
        result += f"{int(days)} day{'s' if days > 1 else ''} "
    if hours > 0:
        result += f"{int(hours)} hour{'s' if hours > 1 else ''}"
    return result.strip()

def format_unix_seconds_to_datetime(unix_seconds):
    date_time = datetime.fromtimestamp(unix_seconds)

    formatted_date_time = date_time.strftime('%Y-%m-%d %H:%M:%S')
    return formatted_date_time

def block_timestamp(): 
    block_number = w3.eth.block_number
    block = w3.eth.get_block(block_number)
    ts = block.timestamp
    return ts

def scaled_days_to_unix_seconds(scaled_days):
    SECONDS_PER_DAY = 86400
    SCALING_FACTOR = 10**18

    scaled_days = scaled_days * 10 ** 18
    days = scaled_days / SCALING_FACTOR
    unix_seconds = days * SECONDS_PER_DAY
    return int(unix_seconds) 

In [615]:
def get_sold():
    sold = contract.functions.totalSupply().call()
    return sold

def get_book_per_cat():
    current_reserve = w3.eth.get_balance(CATS_RESERVE)
    current_supply = contract.functions.adjustedTotalSupply().call()
    book_value_per_cat = current_reserve / current_supply
    return book_value_per_cat

def get_target_sale_time(sold):
    target_sale_time = contract.functions.getTargetSaleTime(sold).call()
    return target_sale_time

def get_vrgda_price(time_since_start, sold):
    vrgda_price = contract.functions.getVRGDAPrice(time_since_start, sold).call()
    return vrgda_price

In [616]:
def to_days_wad_unsafe(x):
    """
    Takes an integer amount of seconds and converts it to a wad amount of days.
    Assumes x is a positive integer, as the function does not handle negative input 
    safely.
    """
    # Constants
    SCALING_FACTOR = 10**18
    SECONDS_PER_DAY = 86400
    
    # Convert seconds to 'wad' scaled days
    scaled_days = (x * SCALING_FACTOR) // SECONDS_PER_DAY
    return scaled_days

In [617]:
start_time = 1710445629

# comment to test
target_time = date_to_unix("2025-01-01")
target_sold = 3234

# uncomment to test
# target_time = block_timestamp()
# target_sold = get_sold()

time_since_start = target_time - start_time

In [618]:
price = get_vrgda_price(to_days_wad_unsafe(time_since_start), target_sold)

In [619]:
print(to_eth(get_book_per_cat()))
print(to_eth(price))

0.9470259
0.94786688


In [620]:
def error_function(time_to_start, target):
    current_price = to_eth(get_book_per_cat())
    predicted_price = to_eth(get_vrgda_price(to_days_wad_unsafe(time_to_start), target))
    return (predicted_price - current_price) / current_price - 0.05

def find_T(time_to_start, T_min, T_max, tolerance=1e-6):
    low = T_min
    high = T_max
    while high - low > tolerance:
        mid = int((low + high) / 2)
        if error_function(time_to_start, mid) * error_function(time_to_start, low) < 0:
            high = mid
        else:
            low = mid
    return int((low + high) / 2)

In [624]:
# Example usage
T_min = 500  # Adjust according to your context
T_max = 4000  # Adjust according to your context
tolerance = 1  # Precision of the solution
T = find_T(time_since_start, T_min, T_max, tolerance)
book = to_eth(get_book_per_cat())
print(f"Approx nr of cats minted to stay close to {book} floor by EOY: {T}")

Approx nr of cats minted to stay close to 0.9470259 floor by EOY: 3240


In [581]:
def unsafe_wad_div(x, y):
    SCALING_FACTOR = 10**18
    try:
        result = (x * SCALING_FACTOR) // y
    except ZeroDivisionError:
        result = 0
    return result

In [589]:
target_sold = 718
per_time_unit = 10e18
result = unsafe_wad_div(target_sold, per_time_unit)

Cat 718 should be sold by: 71.0


In [591]:
sell_by_seconds = scaled_days_to_unix_seconds(result)
target_datetime = start_time + sell_by_seconds
target_date = format_unix_seconds_to_datetime(target_datetime)

print(f"Cat {target_sold} is scheduled to be sold by: {target_date}")

Cat 718 is scheduled to be sold by: 2024-05-24 12:47:09
